In [2]:
import pandas as pd
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules
df = pd.read_excel("online_retail.xlsx", engine="openpyxl")
df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


In [3]:
def outlier_thresholds(dataframe, variable):
    quartile1 = dataframe[variable].quantile(0.01)
    quartile3 = dataframe[variable].quantile(0.99)
    interquantile_range = quartile3 - quartile1
    up_limit = quartile3 + 1.5 * interquantile_range
    low_limit = quartile1 - 1.5 * interquantile_range
    return low_limit, up_limit


def replace_with_thresholds(dataframe, variable):
    low_limit, up_limit = outlier_thresholds(dataframe, variable)
    dataframe.loc[(dataframe[variable] > up_limit), variable] = up_limit

replace_with_thresholds(df, "Quantity")
replace_with_thresholds(df, "UnitPrice")

In [4]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
Quantity,541909.0,8.519687,154.626855,-80995.00,1.00,3.00,10.00,253.000
UnitPrice,541909.0,3.327278,21.652615,-11062.06,1.25,2.08,4.13,44.715
CustomerID,406829.0,15287.690570,1713.600303,12346.00,13953.00,15152.00,16791.00,18287.000


In [5]:
df.dropna(axis=0, inplace=True)
df["Description"] = df["Description"].str.strip(" ")
df = df[~df["InvoiceNo"].str.contains("C", na=False)]
df = df[df["StockCode"].apply(lambda x: str(x).isnumeric())]
df = df[df["Quantity"] > 0]

In [6]:
basket = (df[df["Country"] == "Germany"]
              .groupby(["InvoiceNo", "Description"])["Quantity"]
              .sum().unstack().fillna(0).applymap(lambda x: 1 if x > 0 else 0))
basket.iloc[0:5, 0:5]

Description,10 COLOUR SPACEBOY PEN,12 COLOURED PARTY BALLOONS,12 IVORY ROSE PEG PLACE SETTINGS,12 MESSAGE CARDS WITH ENVELOPES,12 PENCIL SMALL TUBE WOODLAND
InvoiceNo,,,,,
536527,0,0,0,0,0
536840,0,0,0,0,0
536861,0,0,0,0,0
536983,0,0,0,0,0
537197,0,0,0,0,0


In [7]:
frequent_itemsets = apriori(basket, min_support=0.01, use_colnames=True)
frequent_itemsets.head()

,support,itemsets
0,0.024943,(10 COLOUR SPACEBOY PEN)
1,0.022676,(12 PENCIL SMALL TUBE WOODLAND)
2,0.013605,(12 PENCILS SMALL TUBE RED RETROSPOT)
3,0.020408,(12 PENCILS TALL TUBE WOODLAND)
4,0.034014,(3 HOOK HANGER MAGIC GARDEN)


In [8]:
rules = association_rules(frequent_itemsets, metric="support", min_threshold=0.01)
rules.head()

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(10 COLOUR SPACEBOY PEN),(LUNCH BAG APPLE DESIGN),0.024943,0.063492,0.011338,0.454545,7.159091,0.009754,1.716931
1,(LUNCH BAG APPLE DESIGN),(10 COLOUR SPACEBOY PEN),0.063492,0.024943,0.011338,0.178571,7.159091,0.009754,1.187026
2,(SPACEBOY LUNCH BOX),(10 COLOUR SPACEBOY PEN),0.106576,0.024943,0.013605,0.127660,5.117988,0.010947,1.117748
3,(10 COLOUR SPACEBOY PEN),(SPACEBOY LUNCH BOX),0.024943,0.106576,0.013605,0.545455,5.117988,0.010947,1.965533
4,(12 PENCILS TALL TUBE WOODLAND),(ROUND SNACK BOXES SET OF4 WOODLAND),0.020408,0.253968,0.015873,0.777778,3.062500,0.010690,3.357143
